# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import json
import os

# Import API key and configuring gmaps
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I (Weatherpy) results into DataFrame


In [2]:
vacationpy_cities = pd.read_csv('output_data/weatherpy/cities.csv')
vacationpy_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Shepstone,-30.74,30.45,69.76,83,99,11.14,ZA,1603846471
1,Basco,20.45,121.97,81.68,83,100,23.09,PH,1603846471
2,Hermanus,-34.42,19.23,60.01,92,100,1.01,ZA,1603846471
3,Thinadhoo,0.53,72.93,83.73,67,41,11.56,MV,1603846471
4,Grevená,40.08,21.43,46.56,81,100,4.70,GR,1603846471
...,...,...,...,...,...,...,...,...,...
557,Tooele,40.53,-112.30,41.36,48,1,3.36,US,1603846576
558,Strezhevoy,60.73,77.59,10.40,85,20,6.71,RU,1603846576
559,Batagay,67.63,134.63,5.65,93,90,1.90,RU,1603846577
560,Straumen,63.87,11.30,37.94,93,0,8.05,NO,1603846577


### Humidity Heatmap

* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#getting location, converting humidity to float, and getting max humidity
locations = vacationpy_cities[['Lat', 'Lng']]
humidity = vacationpy_cities['Humidity'].astype(float)
max_humidity = humidity.max()

#google maps
fig= gmaps.figure()

#adding heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, 
                                 max_intensity =max_humidity, point_radius = 1)
fig.add_layer(heat_layer)

#printing
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the DataFrame to find your ideal weather condition.
* max temperature <= 80
* min  temperature >66
* Wind speed >= 3 mph.
* Zero cloudiness.
* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [4]:
#new datafram with criterias
new_weather_df = vacationpy_cities[(vacationpy_cities['Max Temp'] >=60) &
                                   (vacationpy_cities['Max Temp'] <= 80) &
                                  (vacationpy_cities['Wind Speed'] >= 3) &
                                  (vacationpy_cities['Cloudiness'] == 0)]
#dropping empty rows
new_weather_df = new_weather_df.dropna(how = 'any')

#printing
new_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,Bilma,18.69,12.92,68.05,25,0,3.31,NE,1603846235
33,Port Elizabeth,-33.92,25.57,62.60,82,0,19.46,ZA,1603846476
41,Villa María,-32.41,-63.24,64.99,65,0,3.00,AR,1603846478
47,Rojhan,28.69,69.95,66.63,25,0,4.76,PK,1603846479
50,Sohag,26.56,31.69,75.20,53,0,9.17,EG,1603846480
56,Serowe,-22.38,26.72,69.26,41,0,7.52,BW,1603846480
62,Jalu,29.03,21.55,70.16,43,0,10.22,LY,1603846481
70,San Quintín,30.48,-115.95,65.62,44,0,6.98,MX,1603846483
77,Grand Gaube,-20.01,57.66,70.00,80,0,4.00,MU,1603846484
101,Chimoio,-19.12,33.48,66.20,100,0,3.36,MZ,1603846489


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#hotel dataframe
hotel_df = new_weather_df
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
21,Bilma,18.69,12.92,68.05,25,0,3.31,NE,1603846235,
33,Port Elizabeth,-33.92,25.57,62.60,82,0,19.46,ZA,1603846476,
41,Villa María,-32.41,-63.24,64.99,65,0,3.00,AR,1603846478,
47,Rojhan,28.69,69.95,66.63,25,0,4.76,PK,1603846479,
50,Sohag,26.56,31.69,75.20,53,0,9.17,EG,1603846480,


In [6]:
#url
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

#parameters
params ={'radius' : 5000,
         'type' : 'lodging',
#          'keyword' : 'Hotel',
         'key': g_key}

missing_counter = 0



for index, row in hotel_df.iterrows():
   
    #getting Lat, and Lng from hotel_df
    lat = row['Lat']
    lng = row['Lng']
    
    #putting lat and lng into params
    params['location'] = f'{lat},{lng}'
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    result = response['results']
    
    #putting hotel name into column if found
    try:
        hotel_df.loc[index, "Hotel Name"] = result[0]['name']
    
    except(KeyError, IndexError):
       missing_counter += 1

#printing
print(f'missing ({missing_counter}) hotel(s)')
hotel_df

missing (2) hotel(s)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
21,Bilma,18.69,12.92,68.05,25,0,3.31,NE,1603846235,Expeditions Ténére voyages
33,Port Elizabeth,-33.92,25.57,62.60,82,0,19.46,ZA,1603846476,39 On Nile Guest House
41,Villa María,-32.41,-63.24,64.99,65,0,3.00,AR,1603846478,Hotel Milenium
47,Rojhan,28.69,69.95,66.63,25,0,4.76,PK,1603846479,NAZIM ALI MAZARI HOME OZMAN
50,Sohag,26.56,31.69,75.20,53,0,9.17,EG,1603846480,Civil Registry - Sohag 1
56,Serowe,-22.38,26.72,69.26,41,0,7.52,BW,1603846480,Hill View Inn
62,Jalu,29.03,21.55,70.16,43,0,10.22,LY,1603846481,Jalu Hotel
70,San Quintín,30.48,-115.95,65.62,44,0,6.98,MX,1603846483,Old Mill Hotel
77,Grand Gaube,-20.01,57.66,70.00,80,0,4.00,MU,1603846484,Veranda Paul et Virginie Hotel & Spa
101,Chimoio,-19.12,33.48,66.20,100,0,3.36,MZ,1603846489,Predio Livraria Ahmed


In [7]:
#dropping missing hotel rows
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
21,Bilma,18.69,12.92,68.05,25,0,3.31,NE,1603846235,Expeditions Ténére voyages
33,Port Elizabeth,-33.92,25.57,62.60,82,0,19.46,ZA,1603846476,39 On Nile Guest House
41,Villa María,-32.41,-63.24,64.99,65,0,3.00,AR,1603846478,Hotel Milenium
47,Rojhan,28.69,69.95,66.63,25,0,4.76,PK,1603846479,NAZIM ALI MAZARI HOME OZMAN
50,Sohag,26.56,31.69,75.20,53,0,9.17,EG,1603846480,Civil Registry - Sohag 1
56,Serowe,-22.38,26.72,69.26,41,0,7.52,BW,1603846480,Hill View Inn
62,Jalu,29.03,21.55,70.16,43,0,10.22,LY,1603846481,Jalu Hotel
70,San Quintín,30.48,-115.95,65.62,44,0,6.98,MX,1603846483,Old Mill Hotel
77,Grand Gaube,-20.01,57.66,70.00,80,0,4.00,MU,1603846484,Veranda Paul et Virginie Hotel & Spa
101,Chimoio,-19.12,33.48,66.20,100,0,3.36,MZ,1603846489,Predio Livraria Ahmed


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))